<a href="https://colab.research.google.com/github/mariskit/Labo2_estructuras/blob/main/Laboratorio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
datos='https://docs.google.com/spreadsheets/d/e/2PACX-1vSNEQuWHBszpu4a-HUkso9g_XVWwGaEPT2PBklRwlM5oVsgDft6tdVW8d8rXzCX3bQCqI9xg2J6U6p-/pub?gid=1362759061&single=true&output=csv'

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import folium

# Cargar los datos del archivo CSV
data = pd.read_csv("archivo.csv")
# Crear un grafo no dirigido
G = nx.Graph()

# Agregar vértices (aeropuertos)
for index, row in data.iterrows():
      G.add_node(row['Source Airport Code'],
                name=row['Source Airport Name'],
                city=row['Source Airport City'],
                country=row['Source Airport Country'],
                latitude=row['Source Airport Latitude'],
                longitude=row['Source Airport Longitude'])
      G.add_node(row['Destination Airport Code'],
                name=row['Destination Airport Name'],
                city=row['Destination Airport City'],
                country=row['Destination Airport Country'],
                latitude=row['Destination Airport Latitude'],
                longitude=row['Destination Airport Longitude'])

  # Función para calcular la distancia entre dos coordenadas geográficas
def haversine(lon1, lat1, lon2, lat2):
      """
      Calcula la distancia haversine entre dos puntos
      en coordenadas geográficas.
      """
      lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
      dlon = lon2 - lon1
      dlat = lat2 - lat1
      a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
      c = 2 * np.arcsin(np.sqrt(a))
      km = 6371 * c
      return km

  # Agregar aristas ponderadas (distancia entre aeropuertos)
for index, row in data.iterrows():
      source_lat = row['Source Airport Latitude']
      source_lon = row['Source Airport Longitude']
      dest_lat = row['Destination Airport Latitude']
      dest_lon = row['Destination Airport Longitude']
      distance = haversine(source_lon, source_lat, dest_lon, dest_lat)
      G.add_edge(row['Source Airport Code'], row['Destination Airport Code'], weight=distance)

  # Coordenadas del centro de Colombia
colombia_center = [4.5709, -74.2973]

  # Crear un mapa con Folium y establecer el zoom inicial en Colombia
m = folium.Map(location=colombia_center, zoom_start=5)

  # Función para generar una tabla HTML con la información del aeropuerto
def generate_html_table(info):
      html = "<table>"
      for key, value in info.items():
          html += "<tr><td><strong>{}</strong></td><td>{}</td></tr>".format(key, value)
      html += "</table>"
      return html

  # Agregar marcadores de aeropuertos al mapa con información en el popup
for node in G.nodes(data=True):
      info = {
          "Código": node[0],
          "Nombre": node[1]['name'],
          "Ciudad": node[1]['city'],
          "País": node[1]['country'],
          "Latitud": node[1]['latitude'],
          "Longitud": node[1]['longitude']
      }
      popup_html = generate_html_table(info)
      folium.Marker([node[1]['latitude'], node[1]['longitude']], popup=popup_html).add_to(m)

  # Mostrar el mapa interactivo
basem=m



  #menu de opciones
def mostrar_menu():
      opciones = ["10 aeropuertos con caminos minimos más largos", "Camino minimo entre dos aeropuertos", "Salir"]
      for i, opcion in enumerate(opciones, start=1):
          print(f"{i}. {opcion}")

def opcion_1():
      print("Has seleccionado la Opción 1. 10 aeropuertos con caminos minimos más largos")
      def longest_shortest_paths(graph, start_node, num_paths=10):
        longest_paths = []
        for target_node in graph.nodes():
            if target_node != start_node:
                try:
                    shortest_path_length = nx.shortest_path_length(graph, start_node, target_node, weight='weight')
                    longest_paths.append((target_node, shortest_path_length))
                except nx.NetworkXNoPath:
                    pass
        longest_paths.sort(key=lambda x: x[1], reverse=True)
        return longest_paths[:num_paths]

      def generate_html_table(data):
        html = "<table>"
        for key, value in data.items():
            html += f"<tr><td>{key}</td><td>{value}</td></tr>"
        html += "</table>"
        return html

      # Obtener el código del aeropuerto de inicio
      start_airport_code = input("Ingrese el código del aeropuerto de inicio: ").upper()

      # Seleccionar el nodo correspondiente al aeropuerto de inicio
      start_node = start_airport_code

      # Obtener los 10 aeropuertos más lejanos desde el nodo seleccionado
      longest_paths = longest_shortest_paths(G, start_node, num_paths=10)

      # Crear un nuevo mapa con Folium y establecer el zoom inicial en Colombia
      m = folium.Map(location=[data[data['Source Airport Code'] == start_node]['Source Airport Latitude'].values[0], data[data['Source Airport Code'] == start_node]['Source Airport Longitude'].values[0]], zoom_start=5)
      # Función para generar una tabla HTML a partir de un diccionario de datos
      def generate_html_table(data):
          html = "<table>"
          for key, value in data.items():
              html += f"<tr><td>{key}</td><td>{value}</td></tr>"
          html += "</table>"
          return html

      # Agregar marcadores de aeropuertos al mapa con información en el popup
      for node in G.nodes(data=True):
          info = {
              "Código": node[0],
              "Nombre": node[1]['name'],
              "Ciudad": node[1]['city'],
              "País": node[1]['country'],
              "Latitud": node[1]['latitude'],
              "Longitud": node[1]['longitude']
          }
          if node[0] == start_node or node[0] in [path[0] for path in longest_paths]:
              popup_html = generate_html_table(info)
              folium.Marker([node[1]['latitude'], node[1]['longitude']], popup=popup_html).add_to(m)

      # Agregar aristas de los caminos mínimos más largos al mapa y calcular la distancia total de cada camino
      for path in longest_paths:
          try:
              path_nodes = nx.shortest_path(G, start_node, path[0])
              path_edges = list(zip(path_nodes[:-1], path_nodes[1:]))
              folium.PolyLine(locations=[(G.nodes[node]['latitude'], G.nodes[node]['longitude']) for node in path_nodes], color='red').add_to(m)
              print(f"Distancia total del camino mínimo a {path[0]}: {path[1]:.2f} km")
          except nx.NetworkXNoPath:
              print(f"No hay camino desde el nodo de inicio {start_node} al nodo {path[0]}")

      # Mostrar el mapa interactivo
      display(m)


def opcion_2():
      print("Has seleccionado la Opción 2. Camino minimo entre dos aeropuertos")
      def shortest_path(graph, start_node, end_node):
          return nx.shortest_path(graph, start_node, end_node, weight='weight')

      # Obtener el código del aeropuerto de inicio
      start_airport = input("Ingrese el código del aeropuerto de inicio: ").upper()

      # Obtener el código del aeropuerto de destino
      end_airport = input("Ingrese el código del aeropuerto de destino: ").upper()

      # Encontrar el camino mínimo entre los aeropuertos de inicio y destino
      shortest_path_nodes = shortest_path(G, start_airport, end_airport)

      # Calcular la distancia total del recorrido
      total_distance = sum(G.edges[node]['weight'] for node in zip(shortest_path_nodes[:-1], shortest_path_nodes[1:]))

      # Crear un nuevo mapa con Folium
      m = folium.Map(location=[G.nodes[start_airport]['latitude'], G.nodes[start_airport]['longitude']], zoom_start=5)

      # Crear un conjunto de nodos en el camino mínimo para verificar más adelante
      shortest_path_set = set(shortest_path_nodes)

      # Agregar marcadores de aeropuertos al mapa con información en el popup
      for node in G.nodes(data=True):
          if node[0] in shortest_path_set:
              info = {
                  "Código": node[0],
                  "Nombre": node[1]['name'],
                  "Ciudad": node[1]['city'],
                  "País": node[1]['country'],
                  "Latitud": node[1]['latitude'],
                  "Longitud": node[1]['longitude']
              }
              popup_html = generate_html_table(info)
              folium.Marker([node[1]['latitude'], node[1]['longitude']], popup=popup_html).add_to(m)

      # Agregar aristas del camino mínimo al mapa
      for i in range(len(shortest_path_nodes) - 1):
          path_edge = (shortest_path_nodes[i], shortest_path_nodes[i+1])
          folium.PolyLine(locations=[(G.nodes[path_edge[0]]['latitude'], G.nodes[path_edge[0]]['longitude']),
                                    (G.nodes[path_edge[1]]['latitude'], G.nodes[path_edge[1]]['longitude'])],
                          color='blue').add_to(m)

      # Mostrar la distancia total del recorrido
      print(f"Distancia total del recorrido: {total_distance:.2f} km")

      # Mostrar el mapa interactivo
      display(m)


def opcion_3():
      print("Has seleccionado la Opción 3. Salir")
      print("Gracias por elegirnos")

def obtener_opcion():
      while True:
          try:
              opcion = int(input("Por favor, elige una opción (1-3): "))
              if opcion < 1 or opcion > 3:
                  print("Por favor, ingresa un número válido (1-3).")
              else:
                  return opcion
          except ValueError:
              print("Por favor, ingresa un número válido (1-3).")

display(basem)
while True:
    mostrar_menu()
    opcion_elegida = obtener_opcion()

    if opcion_elegida == 1:
        opcion_1()
    elif opcion_elegida == 2:
        opcion_2()
    else:
        opcion_3()
        break